# Qwen2.5-7B-Instruct to predict IPV and utilize DetectIPV Paper Evaluation Pipeline
Key tasks:
* Binary IPV detection (IPV vs NOT_IPV)
* Multi-label subtype detection (Physical, Emotional, Sexual)

**Key Evaluation Components from the Paper**
| Figure/Table | What it Shows |
|---------------|----------------|
| **Fig. 2(a)** | ROC curve for *General Violence Model* |
| **Fig. 2(b–c)** | “Waterfall” plot: sentences sorted by predicted confidence; color = ground-truth subtype |
| **Fig. 3(a)** | AUROC of Type-Specific vs General Models |
| **Fig. 3(b–e)** | More waterfall plots for type-specific models |
| **Fig. 4** | Radial visualization of confidence in 3-D type space |
| **Table 4** | AUROC for models trained on different negative examples |

## 1. Prompt Design

To evaluate how prompt structure influences LLM performance on IPV detection, I used **five prompting strategies** for both binary (IPV vs. NOT_IPV) and multi-label (Physical, Emotional, Sexual) tasks.  
This allows testing how reasoning depth, context, and self-reflection affect accuracy, AUC, and interpretability.

| Prompt Type | Description | Advantage |
|--------------|-------------|------------|
| **Zero-Shot** | Direct question without examples or reasoning. | Tests model’s innate understanding of IPV cues. |
| **Few-Shot** | Includes short examples before prediction. | Leverages in-context learning and improves clarity. |
| **Chain-of-Thought** | Prompts reasoning steps before final label. | Makes predictions more explainable and robust. |
| **Meta-Prompt** | Frames model as an IPV analyst with reflective reasoning. | Encourages careful, context-aware decisions. |
| **Self-Consistency** | Runs multiple internal judgments and votes. | Improves stability and reduces random errors. |

These prompt variants help identify which instruction style yields the **most reliable and interpretable results** for both general and type-specific IPV detection.

### 1.1 Binary IPV Detection Prompts

In [1]:
# (1) Zero-Shot — final (no inline JSON examples)
prompt_binary_zeroshot = """
You are analyzing whether a sentence describes Intimate Partner Violence (IPV).

Definition:
Intimate Partner Violence includes any form of physical, emotional, or sexual harm
inflicted by a partner. This may involve control, manipulation, threats, humiliation,
or physical aggression.

Decide if the following sentence involves IPV or not.

Sentence: "{text}"

Return ONLY one JSON object, enclosed between <json> and </json>.
Format: a single key "label" with value "IPV" or "NOT_IPV".
Print nothing else.
""".strip()


# (2) Few-Shot — final (no inline JSON examples)
prompt_binary_fewshot = """
You are classifying sentences as describing Intimate Partner Violence (IPV) or NOT_IPV.

Definition:
IPV occurs when one partner harms or controls another physically, emotionally, or sexually.

Here are examples (for guidance only):
1. "He pushed me against the wall and shouted that I’m worthless." → IPV
2. "She often insults me and forbids me from seeing my friends." → IPV
3. "We argued but both apologized and moved on peacefully." → NOT_IPV
4. "He surprised me with flowers after work." → NOT_IPV

Now analyze this sentence:
"{text}"

Return ONLY one JSON object, enclosed between <json> and </json>.
Format: {"label": "IPV"} or {"label": "NOT_IPV"}.
Print nothing else.
""".strip()


# (3) Chain-of-Thought — final (no inline JSON examples)
prompt_binary_cot = """
You are reasoning step-by-step to decide if a sentence describes Intimate Partner Violence (IPV).

Guidelines:
- IPV includes threats, coercion, physical harm, humiliation, or emotional manipulation.
- NOT_IPV describes healthy, neutral, or unrelated situations.

Think internally (do NOT show your reasoning) about:
1. Does the sentence show any behavior that causes harm, fear, or control?
2. Is there a partner/relationship context?
3. Does it express affection or support instead of harm?

After thinking silently, return ONLY one JSON object, enclosed between <json> and </json>.
Format: {"label": "IPV"} or {"label": "NOT_IPV"}.
Print nothing else.

Sentence: "{text}"
""".strip()


# (4) Meta — final (no inline JSON examples)
prompt_binary_meta = """
You are a social-behavioral analyst evaluating sentences for signs of Intimate Partner Violence (IPV).

Your objective is to be accurate but cautious.
- If the sentence clearly involves harm, coercion, or control → label as IPV.
- If the sentence shows affection, neutrality, or uncertainty → label as NOT_IPV.

Reflect internally before answering; do NOT print your reasoning.

Sentence: "{text}"

Return ONLY one JSON object, enclosed between <json> and </json>.
Format: {"label": "IPV"} or {"label": "NOT_IPV"}.
Print nothing else.
""".strip()


# (5) Self-Consistency — final (no inline JSON examples)
prompt_binary_selfconsistency = """
You will internally evaluate the sentence for Intimate Partner Violence (IPV) multiple times
and choose the majority label as your final answer.

Guidelines:
- IPV → signs of physical, emotional, or sexual harm, threats, or coercion.
- NOT_IPV → supportive, neutral, or unrelated content.

Do NOT reveal thoughts or votes.

Sentence: "{text}"

Return ONLY one JSON object, enclosed between <json> and </json>.
Format: {"label": "IPV"} or {"label": "NOT_IPV"}.
Print nothing else.
""".strip()

### 1.2 Multi-label IPV Type Detection Prompts

In [26]:
# (1) Zero-Shot — strict JSON output (no example line)
prompt_multilabel_zeroshot = """
You are classifying a sentence for Intimate Partner Violence (IPV) subtypes.

Valid labels (choose any subset): Physical, Emotional, Sexual, NOT_IPV.
If the sentence does not describe IPV, include only NOT_IPV.
If it shows multiple IPV types, include all that apply.

Return EXACTLY one JSON object wrapped in <json> and </json>.
Use a single key "labels" whose value is a list drawn ONLY from:
["Physical", "Emotional", "Sexual", "NOT_IPV"].
Do not include any other keys or text.

Sentence: "{text}"
""".strip()


# (2) Few-Shot — guided, strict JSON output (no example line)
prompt_multilabel_fewshot = """
You are classifying a sentence for Intimate Partner Violence (IPV) subtypes.

Valid labels (choose any subset): Physical, Emotional, Sexual, NOT_IPV.

Guidance (for understanding only — do not copy these into your output):
- Physical: hitting, pushing, choking, restraining, use or threat of physical force.
- Emotional: humiliation, manipulation, isolation, threats, control, verbal abuse.
- Sexual: coercion, unwanted sexual acts, pressure, harassment.
- NOT_IPV: ordinary disagreement or neutral statement without violence or coercion.

If the sentence shows multiple IPV types, include all.
If it shows none, include only NOT_IPV.

Return EXACTLY one JSON object wrapped in <json> and </json>.
Use a single key "labels" whose value is a list drawn ONLY from:
["Physical", "Emotional", "Sexual", "NOT_IPV"].
Print nothing else.

Sentence: "{text}"
""".strip()


# (3) Chain-of-Thought — silent reasoning, strict JSON output (no example line)
prompt_multilabel_cot = """
Decide which IPV subtype(s) apply to the sentence.
Think silently and do not reveal your reasoning.

Valid labels: Physical, Emotional, Sexual, NOT_IPV.
If none apply, include only NOT_IPV.

Return EXACTLY one JSON object wrapped in <json> and </json>.
Use a single key "labels" whose value is a list drawn ONLY from:
["Physical", "Emotional", "Sexual", "NOT_IPV"].
Print nothing else.

Sentence: "{text}"
""".strip()


# (4) Meta — expert framing, strict JSON output (no example line)
prompt_multilabel_meta = """
You are an expert on Intimate Partner Violence (IPV) classification.
Reflect internally; do not show your reasoning.

Valid labels: Physical, Emotional, Sexual, NOT_IPV.
If none apply, include only NOT_IPV.

Return EXACTLY one JSON object wrapped in <json> and </json>.
Use a single key "labels" whose value is a list drawn ONLY from:
["Physical", "Emotional", "Sexual", "NOT_IPV"].
Print nothing else.

Sentence: "{text}"
""".strip()


# (5) Self-Consistency — internal deliberation, strict JSON output (no example line)
prompt_multilabel_selfconsistency = """
Evaluate the sentence multiple times INTERNALLY and output a stable, consistent final set of labels.

Valid labels: Physical, Emotional, Sexual, NOT_IPV.
If none apply, include only NOT_IPV.

Return EXACTLY one JSON object wrapped in <json> and </json>.
Use a single key "labels" whose value is a list drawn ONLY from:
["Physical", "Emotional", "Sexual", "NOT_IPV"].
Print nothing else.

Sentence: "{text}"
""".strip()

## 2. System & Model Setup

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import json
import os
import time
from datetime import datetime

In [4]:
#FILENAMES
model_name = "Qwen/Qwen2.5-7B-Instruct"

#Load Model & Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

## 3. Generate Predictions

In [5]:
#Clone from git
!git clone https://github.com/zelaneroz/ipvresearch25
%cd ipvresearch25/1_LLM_Eval

Cloning into 'ipvresearch25'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 39 (delta 16), reused 22 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 5.54 MiB | 21.66 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/ipvresearch25/1_LLM_Eval


In [6]:
#Load dataset
filename = "../Dataset/617points.csv"
df = pd.read_csv(filename)
print("Loaded dataset with", len(df), "rows")
df.head()

Loaded dataset with 618 rows


,Unnamed: 0,items,Physical Abuse,Emotional Abuse,Sexual Abuse,Tag,type
0,0,I'm sitting here with a goofy smile and feelin...,False,False,False,False,soft
1,1,It's really boosting my confidence when he say...,False,False,False,False,soft
2,2,I never imagined that someone could make me s...,False,False,False,False,soft
3,3,He motivates me to become the best version of ...,False,False,False,False,soft
4,4,He’s like a best friend that I can also live w...,False,False,False,False,soft


In [27]:
#Prompt dictionaries
binary_prompts = {
    "zeroshot": prompt_binary_zeroshot,
    "fewshot": prompt_binary_fewshot,
    "cot": prompt_binary_cot,
    "meta": prompt_binary_meta,
    "selfconsistency": prompt_binary_selfconsistency
}

multilabel_prompts = {
    "zeroshot": prompt_multilabel_zeroshot,
    "fewshot": prompt_multilabel_fewshot,
    "cot": prompt_multilabel_cot,
    "meta": prompt_multilabel_meta,
    "selfconsistency": prompt_multilabel_selfconsistency
}

In [28]:
def run_llm(prompt_text):
    """Feed a prompt into the LLM and return only the generated portion (not echoed prompt)."""
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=64,
        temperature=0.0,   # deterministic
        do_sample=False
    )
    gen_tokens = output[0][inputs["input_ids"].shape[-1]:]
    text = tokenizer.decode(gen_tokens, skip_special_tokens=True)
    return text.strip()

In [16]:
def test_binary_prompts(df, n_samples=3):
    """
    Run and test all binary prompt types.
    Extracts IPV / NOT_IPV from within <json>...</json> blocks
    and saves each prompt type's outputs to a separate JSON file.
    """
    from datetime import datetime
    from pathlib import Path
    import json
    import re

    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    results_dir = Path("../1_LLM_Eval/test_results")
    results_dir.mkdir(parents=True, exist_ok=True)

    df_subset = df.head(n_samples)
    print("Running binary classification tests...")
    print(f"Number of samples: {len(df_subset)}")
    print(f"Results will be saved in: {results_dir}\n")

    for prompt_type, template in binary_prompts.items():
        print(f"Testing prompt type: {prompt_type}")

        records = []

        for i, row in df_subset.iterrows():
            text = row["items"] if "items" in df.columns else str(row.iloc[0])
            prompt_text = template.replace("{text}", text)

            try:
                result_text = run_llm(prompt_text)
            except Exception as e:
                result_text = f"ERROR: {e}"

            # Extract IPV/NOT_IPV from <json>...</json>
            match = re.search(r"<json[^>]*>\s*(.*?)\s*</json>", result_text, re.DOTALL | re.IGNORECASE)
            label = None
            if match:
                block = match.group(1).strip()
                # Try to load as JSON
                try:
                    parsed = json.loads(block)
                    label = parsed.get("label") or parsed.get("labels")
                except json.JSONDecodeError:
                    # Fallback: detect plain string like NOT_IPV or IPV
                    if "NOT_IPV" in block.upper():
                        label = "NOT_IPV"
                    elif "IPV" in block.upper():
                        label = "IPV"
            else:
                # Fallback for any text outside JSON
                if "NOT_IPV" in result_text.upper():
                    label = "NOT_IPV"
                elif "IPV" in result_text.upper(o):
                    label = "IPV"

            record = {
                "id": int(i),
                "prompt_type": prompt_type,
                "extracted_label": label or "UNKNOWN"
            }
            records.append(record)

        # Save per prompt type
        output_path = results_dir / f"binary_{prompt_type}.json"
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(records, f, indent=4, ensure_ascii=False)

        print(f"Saved results for '{prompt_type}' to {output_path}")

    print("\nAll binary prompt tests completed.")

In [ ]:
def test_multilabel_prompts(df, n_samples=3):
    """
    Run and test all multilabel prompt types.
    Extracts Physical / Emotional / Sexual / NOT_IPV labels from <json>...</json> blocks
    and saves each prompt type's outputs to a separate JSON file.
    """
    import re
    import json
    from datetime import datetime
    from pathlib import Path

    IPV_TYPES = ["Physical", "Emotional", "Sexual", "NOT_IPV"]

    # Prepare subset and output directory
    df_subset = df.head(n_samples)
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    results_dir = Path("../1_LLM_Eval/test_results")
    results_dir.mkdir(parents=True, exist_ok=True)

    print("Running multilabel classification tests...")
    print(f"Number of samples: {len(df_subset)}")
    print(f"Results will be saved in: {results_dir}\n")

    # Iterate through multilabel prompt types
    for prompt_type, template in multilabel_prompts.items():
        print(f"Testing prompt type: {prompt_type}")

        records = []

        for i, row in df_subset.iterrows():
            # Get text input
            text = row["items"] if "items" in df.columns else str(row.iloc[0])
            prompt_text = template.replace("{text}", text)

            # Run model
            try:
                inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
                output = model.generate(
                    **inputs,
                    max_new_tokens=256,
                    temperature=0.0,
                    do_sample=False
                )
                gen_tokens = output[0][inputs["input_ids"].shape[-1]:]
                result_text = tokenizer.decode(gen_tokens, skip_special_tokens=True)
            except Exception as e:
                result_text = f"ERROR: {e}"

            # Extract content within <json>...</json>
            labels = []
            match = re.search(r"<json[^>]*>\s*(.*?)\s*</json>", result_text, re.DOTALL | re.IGNORECASE)

            if match:
                block = match.group(1).strip()
                try:
                    parsed = json.loads(block)

                    # Handle dict format
                    if isinstance(parsed, dict):
                        val = parsed.get("labels") or parsed.get("label")
                        if val:
                            if isinstance(val, list):
                                labels = [v for v in val if v in IPV_TYPES]
                            elif isinstance(val, str) and val in IPV_TYPES:
                                labels = [val]

                    # Handle direct list format
                    elif isinstance(parsed, list):
                        labels = [v for v in parsed if v in IPV_TYPES]

                except json.JSONDecodeError:
                    # Fallback: scan text manually if JSON fails
                    for t in IPV_TYPES:
                        if t.lower() in block.lower():
                            labels.append(t)
            else:
                # Fallback: scan the entire text if no <json> tags
                for t in IPV_TYPES:
                    if t.lower() in result_text.lower():
                        labels.append(t)

            labels = sorted(set(labels)) or ["UNKNOWN"]

            record = {
                "id": int(i),
                "prompt_type": prompt_type,
                "extracted_labels": labels
            }
            records.append(record)

        # Save per prompt type
        output_path = results_dir / f"multilabel_{prompt_type}.json"
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(records, f, indent=4, ensure_ascii=False)

        print(f"Saved results for '{prompt_type}' to {output_path}")

    print("\nAll multilabel prompt tests completed.")

## 4. Evaluation Pipeline

1. **Prompt Comparison Phase**
   * ~~Design multiple prompt variants for each task:~~
     - ~~**Binary Prompts:** Different phrasings for IPV vs NOT_IPV classification.~~
     - ~~**Multi-label Prompts:** Different phrasings for type-specific labeling (Physical, Emotional, Sexual, None).~~
   * ~~Run all prompts on the same dataset.~~
   * Compute for each prompt:
     - Accuracy, F1-score, ROC-AUC (per task or per subtype)
     - Average confidence calibration (mean predicted probability for positives/negatives)
   * Visualize:
     - **Prompt Comparison Table or Bar Plot** showing AUC/F1 across prompts
     - Identify the **best-performing prompt** for each task (Binary, Multi-label)

2. **Final Evaluation with Best Prompt**
   * **Binary (General IPV Model):**
     - Use the best binary prompt to compute final **ROC curve**, **AUC**, **Precision-Recall**, **Accuracy**, and **F1-score**.
     - Plot **ROC Curve** (replicating Fig. 2a).
     - Generate **Waterfall Plot** — sentences sorted by prediction confidence, color-coded by true label (Physical, Emotional, Sexual, Negative).

   * **Multi-label (Type-Specific Models):**
     - Use the best multi-label prompt to compute **per-type ROC/AUC** for Physical, Emotional, and Sexual abuse.
     - Compare type-specific vs. general model performance (bar chart similar to Fig. 3a).
     - Produce **Waterfall Plots** for each subtype showing confidence distribution and overlap between true labels and predictions.

3. **(Optional) Extended Visualization**
   * Create a **3D or Radial Plot** showing confidence magnitudes of all three type-specific predictions for interpretability.